In [2]:
import pandas as pd

taula = pd.read_pickle('./data/processed/dataframe.pkl')

from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

In [3]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,Creatinina >1.5 binària,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,0,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,0,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,0,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,0,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,0,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


In [4]:
AMB_PA = taula[taula["PA diagnosticada"] == 1.0]
AMB_PA_MECVV = taula[
    (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] < 30)]
SENSE_PA = taula[
           (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] > 30) &
           (taula['P diagnosticada'] == 1.0)]
TOTAL = taula

# Síndromes geriàtrics

## Index de Charlson

In [5]:
Mitjana_charlson = AMB_PA['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 10.24 ± 10.73


In [6]:
Mitjana_charlson = AMB_PA_MECVV['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA_MECVV['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 16.00 ± 11.60


In [7]:
Mitjana_charlson = SENSE_PA['Charlson'].mean()
desviacio_estandard_charlson = SENSE_PA['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 16.50 ± 12.83


In [8]:
Mitjana_charlson = TOTAL['Charlson'].mean()
desviacio_estandard_charlson = TOTAL['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 11.62 ± 13.16


#### Comparació distributiva

##### Normalitat i p-valor

In [9]:
stat, p_value = shapiro(AMB_PA['Charlson'])
stat, p_value = shapiro(AMB_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7994
Valor p = 0.0020
El nombre de files no segueix una distribució normal.


In [10]:
stat, p_value = shapiro(AMB_PA_MECVV['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9651
Valor p = 0.8571
El nombre de files segueix una distribució normal


In [11]:
stat, p_value = shapiro(SENSE_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9177
Valor p = 0.4113
El nombre de files segueix una distribució normal


In [12]:
stat, p_value = shapiro(TOTAL['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7836
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [13]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Charlson'], AMB_PA_MECVV['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.23085687112868614


In [14]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Charlson'], SENSE_PA['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.1977214805403812


In [15]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['Charlson'], SENSE_PA['Charlson'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.9360013644802581


In [16]:
stat, p_value_14 = mannwhitneyu(AMB_PA['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.8623665948573777


In [17]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.2016616772763471


In [18]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.19370672249585386


## Caigudes prèvies

In [19]:
caprev_AMB_PA= (AMB_PA['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA:.2f}%")

Percentatge de caigudes prèvies: 5.88%


In [20]:
caprev_AMB_PA_MECVV= (AMB_PA_MECVV['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA_MECVV:.2f}%")

Percentatge de caigudes prèvies: 0.00%


In [21]:
caprev_SENSE_PA= (SENSE_PA['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_SENSE_PA:.2f}%")

Percentatge de caigudes prèvies: 12.50%


In [22]:
caprev_TOTAL= (TOTAL['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_TOTAL:.2f}%")

Percentatge de caigudes prèvies: 6.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [23]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=AMB_PA_MECVV['Caigudes prèvies'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.


In [24]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=SENSE_PA['Caigudes prèvies'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.


In [25]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Caigudes prèvies'], columns=SENSE_PA['Caigudes prèvies'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [26]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 3.7353515625
P-valor entre AMB_PA i TOTAL: 0.053272454982386926
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.


In [27]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.


In [28]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.


## Deliris

In [29]:
del_AMB_PA= (AMB_PA['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA:.2f}%")

Percentatge de deliris: 23.53%


In [30]:
del_AMB_PA_MECVV= (AMB_PA_MECVV['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA_MECVV:.2f}%")

Percentatge de deliris: 25.00%


In [31]:
del_SENSE_PA= (SENSE_PA['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_SENSE_PA:.2f}%")

Percentatge de deliris: 62.50%


In [32]:
del_TOTAL= (TOTAL['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_TOTAL:.2f}%")

Percentatge de deliris: 20.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [33]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Deliris'], columns=AMB_PA_MECVV['Deliris'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.


In [34]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Deliris'], columns=SENSE_PA['Deliris'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.


In [35]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Deliris'], columns=SENSE_PA['Deliris'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [36]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Deliris'], columns=TOTAL['Deliris'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.


In [37]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Deliris'], columns=TOTAL['Deliris'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.


In [38]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Deliris'], columns=TOTAL['Deliris'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.


## Demència

In [39]:
dem_AMB_PA= (AMB_PA['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_AMB_PA:.2f}%")

Percentatge de demència: 47.06%


In [40]:
dem_AMB_PA_MECVV= (AMB_PA_MECVV['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_AMB_PA_MECVV:.2f}%")

Percentatge de demència: 25.00%


In [41]:
dem_SENSE_PA= (SENSE_PA['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_SENSE_PA:.2f}%")

Percentatge de demència: 62.50%


In [42]:
dem_TOTAL= (TOTAL['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_TOTAL:.2f}%")

Percentatge de demència: 32.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [43]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Demència'], columns=AMB_PA_MECVV['Demència'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.


In [44]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Demència'], columns=SENSE_PA['Demència'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.


In [45]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Demència'], columns=SENSE_PA['Demència'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [46]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Demència'], columns=TOTAL['Demència'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 13.223042052469136
P-valor entre AMB_PA i TOTAL: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.


In [47]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Demència'], columns=TOTAL['Demència'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.


In [48]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Demència'], columns=TOTAL['Demència'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.


## Síndrome depressiu

In [49]:
depre_AMB_PA= (AMB_PA['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA:.2f}%")

Percentatge de síndrome depressiu: 23.53%


In [50]:
depre_AMB_PA_MECVV= (AMB_PA_MECVV['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA_MECVV:.2f}%")

Percentatge de síndrome depressiu: 12.50%


In [51]:
depre_SENSE_PA= (SENSE_PA['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_SENSE_PA:.2f}%")

Percentatge de síndrome depressiu: 12.50%


In [52]:
depre_TOTAL= (TOTAL['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_TOTAL:.2f}%")

Percentatge de síndrome depressiu: 14.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [53]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=AMB_PA_MECVV['Síndrome depressiu'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.9600000000000002
P-valor entre AMB_PA i AMB_PA_MECVV: 0.32718687779030275
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.


In [54]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=SENSE_PA['Síndrome depressiu'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.


In [55]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Síndrome depressiu'], columns=SENSE_PA['Síndrome depressiu'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [56]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.


In [57]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 1.469387755102041
P-valor entre AMB_PA_MECVV i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.


In [58]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.


## Incontinència urinària

In [59]:
ic_AMB_PA= (AMB_PA['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_AMB_PA:.2f}%")

Percentatge d'incontinència urinària: 29.41%


In [60]:
ic_AMB_PA_MECVV= (AMB_PA_MECVV['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_AMB_PA_MECVV:.2f}%")

Percentatge d'incontinència urinària: 50.00%


In [61]:
ic_SENSE_PA= (SENSE_PA['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_SENSE_PA:.2f}%")

Percentatge d'incontinència urinària: 0.00%


In [62]:
ic_TOTAL= (TOTAL['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_TOTAL:.2f}%")

Percentatge d'incontinència urinària: 16.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [63]:
stat, p_value = shapiro(AMB_PA['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [64]:
stat, p_value = shapiro(AMB_PA_MECVV['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [65]:
stat, p_value = shapiro(SENSE_PA['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [66]:
stat, p_value = shapiro(TOTAL['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4412
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [67]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=AMB_PA_MECVV['Incont.uri'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.6666666666666665
P-valor entre AMB_PA i AMB_PA_MECVV: 0.10247043485974942
No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.


In [68]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=SENSE_PA['Incont.uri'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.


In [69]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Incont.uri'], columns=SENSE_PA['Incont.uri'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [70]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 12.524513888888887
P-valor entre AMB_PA i TOTAL: 0.00040164738150225485
Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.


In [71]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.


In [72]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.


## Incontinència fecal 

In [73]:
ifec_AMB_PA= (AMB_PA['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_AMB_PA:.2f}%")

Percentatge d'incontinència fecal: 29.41%


In [74]:
ifec_AMB_PA_MECVV= (AMB_PA_MECVV['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_AMB_PA_MECVV:.2f}%")

Percentatge d'incontinència fecal: 25.00%


In [75]:
ifec_SENSE_PA= (SENSE_PA['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_SENSE_PA:.2f}%")

Percentatge d'incontinència fecal: 0.00%


In [76]:
ifec_TOTAL= (TOTAL['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_TOTAL:.2f}%")

Percentatge d'incontinència fecal: 12.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [77]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=AMB_PA_MECVV['Incont.fec'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.


In [78]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=SENSE_PA['Incont.fec'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.


In [79]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Incont.fec'], columns=SENSE_PA['Incont.fec'])

chi2_23, p_value_23, dof_23, expected_13 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [80]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 12.524513888888887
P-valor entre AMB_PA i TOTAL: 0.00040164738150225485
Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.


In [81]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.


In [82]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.


## Úlceres de pressió

In [83]:
up_AMB_PA= (AMB_PA['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA:.2f}%")

Percentatge de úlceres de pressió: 5.88%


In [84]:
up_AMB_PA_MECVV= (AMB_PA_MECVV['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA_MECVV:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [85]:
up_SENSE_PA= (SENSE_PA['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_SENSE_PA:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [86]:
up_TOTAL= (TOTAL['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_TOTAL:.2f}%")

Percentatge de úlceres de pressió: 2.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [87]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=AMB_PA_MECVV['Úlceres pressió'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.")


Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.


In [88]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=SENSE_PA['Úlceres pressió'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.")


Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.


In [89]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Úlceres pressió'], columns=SENSE_PA['Úlceres pressió'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [90]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.")


Estadístic xi-quadrat entre AMB_PA i TOTAL: 3.7353515625
P-valor entre AMB_PA i TOTAL: 0.053272454982386926
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.


In [91]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.")


Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.


In [92]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.")


Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.


## Immobilitat

In [93]:
imm_AMB_PA= (AMB_PA['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA:.2f}%")

Percentatge d'immobilitat : 0.00%


In [94]:
imm_AMB_PA_MECVV= (AMB_PA_MECVV['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA_MECVV:.2f}%")

Percentatge d'immobilitat : 0.00%


In [95]:
imm_SENSE_PA= (SENSE_PA['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_SENSE_PA:.2f}%")

Percentatge d'immobilitat : 0.00%


In [96]:
imm_TOTAL= (TOTAL['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_TOTAL:.2f}%")

Percentatge d'immobilitat : 0.00%


#### Comparació distributiva

##### Normalitat i p-valor 

In [97]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Immobilitat'], columns=AMB_PA_MECVV['Immobilitat']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i AMB_PA_MECVV.


In [98]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Immobilitat'], columns=SENSE_PA['Immobilitat']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i SENSE_PA.


In [99]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Immobilitat'], columns=SENSE_PA['Immobilitat']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i SENSE_PA.") 


ValueError: No data; `observed` has size 0.

In [100]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Immobilitat'], columns=TOTAL['Immobilitat']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i TOTAL.


In [101]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Immobilitat'], columns=TOTAL['Immobilitat']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i TOTAL.


In [102]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Immobilitat'], columns=TOTAL['Immobilitat']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en SENSE_PA i TOTAL.


## Confusió

In [103]:
conf_AMB_PA= (AMB_PA['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA:.2f}%")

Percentatge de confusió : 23.53%


In [104]:
conf_AMB_PA_MECVV= (AMB_PA_MECVV['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA_MECVV:.2f}%")

Percentatge de confusió : 25.00%


In [105]:
conf_SENSE_PA= (SENSE_PA['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_SENSE_PA:.2f}%")

Percentatge de confusió : 62.50%


In [106]:
conf_TOTAL= (TOTAL['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_TOTAL:.2f}%")

Percentatge de confusió : 20.00%


#### Comparació distributiva

##### Normalitat i p-valor 

In [107]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Confusió'], columns=AMB_PA_MECVV['Confusió']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i AMB_PA_MECVV.


In [108]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Confusió'], columns=SENSE_PA['Confusió']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i SENSE_PA.


In [109]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Confusió'], columns=SENSE_PA['Confusió']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [110]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Confusió'], columns=TOTAL['Confusió']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i TOTAL.


In [111]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Confusió'], columns=TOTAL['Confusió']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i TOTAL.


In [112]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Confusió'], columns=TOTAL['Confusió']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de confusió en SENSE_PA i TOTAL.


## Osteoporosi

In [113]:
ost_AMB_PA= (AMB_PA['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [114]:
ost_AMB_PA_MECVV= (AMB_PA_MECVV['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA_MECVV:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [115]:
ost_SENSE_PA= (SENSE_PA['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_SENSE_PA:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [116]:
ost_TOTAL= (TOTAL['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_TOTAL:.2f}%")

Percentatge d'osteoporosi: 0.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [117]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Osteoporosi'], columns=AMB_PA_MECVV['Osteoporosi']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i AMB_PA_MECVV.


In [118]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Osteoporosi'], columns=SENSE_PA['Osteoporosi']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i SENSE_PA.


In [119]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Osteoporosi'], columns=SENSE_PA['Osteoporosi']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi  en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [120]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Osteoporosi'], columns=TOTAL['Osteoporosi']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i TOTAL.


In [121]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Osteoporosi'], columns=TOTAL['Osteoporosi']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i TOTAL.


In [122]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Osteoporosi'], columns=TOTAL['Osteoporosi']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en SENSE_PA i TOTAL.


## Sarcopènia

In [123]:
sarc_AMB_PA= (AMB_PA['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA:.2f}%")

Percentatge de sarcopènia: 0.00%


In [124]:
sarc_AMB_PA_MECVV= (AMB_PA_MECVV['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA_MECVV:.2f}%")

Percentatge de sarcopènia: 0.00%


In [125]:
sarc_SENSE_PA= (SENSE_PA['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_SENSE_PA:.2f}%")

Percentatge de sarcopènia: 12.50%


In [126]:
sarc_TOTAL= (TOTAL['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_TOTAL:.2f}%")

Percentatge de sarcopènia: 4.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [127]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Sarcopènia'], columns=AMB_PA_MECVV['Sarcopènia']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i AMB_PA_MECVV.


In [128]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Sarcopènia'], columns=SENSE_PA['Sarcopènia']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia AMB_PA i SENSE_PA.


In [129]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Sarcopènia'], columns=SENSE_PA['Sarcopènia']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [130]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Sarcopènia'], columns=TOTAL['Sarcopènia']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i TOTAL.


In [131]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Sarcopènia'], columns=TOTAL['Sarcopènia']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i TOTAL.


In [132]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Sarcopènia'], columns=TOTAL['Sarcopènia']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de sarcopènia en SENSE_PA i TOTAL.


## Trastorns del son

In [133]:
ts_AMB_PA= (AMB_PA['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


In [134]:
ts_AMB_PA_MECVV= (AMB_PA_MECVV['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA_MECVV:.2f}%")

Percentatge de trastorns del son: 0.00%


In [135]:
ts_SENSE_PA= (SENSE_PA['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_SENSE_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


In [136]:
ts_TOTAL= (TOTAL['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_TOTAL:.2f}%")

Percentatge de trastorns del son: 0.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [137]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Probl.son'], columns=AMB_PA_MECVV['Probl.son']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i AMB_PA_MECVV.


In [138]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Probl.son'], columns=SENSE_PA['Probl.son']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i SENSE_PA.


In [139]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Probl.son'], columns=SENSE_PA['Probl.son']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [140]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Probl.son'], columns=TOTAL['Probl.son']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i TOTAL.


In [141]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Probl.son'], columns=TOTAL['Probl.son']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i TOTAL.


In [142]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Probl.son'], columns=TOTAL['Probl.son']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en SENSE_PA i TOTAL.


## Dolor crònic

In [143]:
dc_AMB_PA= (AMB_PA['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


In [144]:
dc_AMB_PA_MECVV= (AMB_PA_MECVV['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA_MECVV:.2f}%")

Percentatge de dolor crònic: 0.00%


In [145]:
dc_SENSE_PA= (SENSE_PA['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_SENSE_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


In [146]:
dc_TOTAL= (TOTAL['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_TOTAL:.2f}%")

Percentatge de dolor crònic: 0.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [147]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Dolor crònic'], columns=AMB_PA_MECVV['Dolor crònic']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i AMB_PA_MECVV.


In [148]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Dolor crònic'], columns=SENSE_PA['Dolor crònic']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i SENSE_PA.


In [149]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Dolor crònic'], columns=SENSE_PA['Dolor crònic']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [150]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Dolor crònic'], columns=TOTAL['Dolor crònic']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i TOTAL.


In [151]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Dolor crònic'], columns=TOTAL['Dolor crònic']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i TOTAL.


In [152]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Dolor crònic'], columns=TOTAL['Dolor crònic']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en SENSE_PA i TOTAL.


## Iatrogènic

In [153]:
iatr_AMB_PA= (AMB_PA['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


In [154]:
iatr_AMB_PA_MECVV= (AMB_PA_MECVV['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA_MECVV:.2f}%")

Percentatge de iatrogènic: 0.00%


In [155]:
iatr_SENSE_PA= (SENSE_PA['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_SENSE_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


In [156]:
iatr_TOTAL= (TOTAL['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_TOTAL:.2f}%")

Percentatge de iatrogènic: 0.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [157]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Iatrogènic'], columns=AMB_PA_MECVV['Iatrogènic']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i AMB_PA_MECVV.


In [158]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Iatrogènic'], columns=SENSE_PA['Iatrogènic']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i SENSE_PA.


In [159]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Iatrogènic'], columns=SENSE_PA['Iatrogènic']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [160]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Iatrogènic'], columns=TOTAL['Iatrogènic']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i TOTAL.


In [161]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Iatrogènic'], columns=TOTAL['Iatrogènic']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i TOTAL.


In [162]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Iatrogènic'], columns=TOTAL['Iatrogènic']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en SENSE_PA i TOTAL.


## Restrenyiment

In [163]:
est_AMB_PA= (AMB_PA['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


In [164]:
est_AMB_PA_MECVV= (AMB_PA_MECVV['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA_MECVV:.2f}%")

Percentatge de restrenyiment: 0.00%


In [165]:
est_SENSE_PA= (SENSE_PA['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_SENSE_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


In [166]:
est_TOTAL= (TOTAL['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_TOTAL:.2f}%")

Percentatge de restrenyiment: 0.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [167]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Restrenyiment'], columns=AMB_PA_MECVV['Restrenyiment']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i AMB_PA_MECVV.


In [168]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Restrenyiment'], columns=SENSE_PA['Restrenyiment']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i SENSE_PA.


In [169]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Restrenyiment'], columns=SENSE_PA['Restrenyiment']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [170]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Restrenyiment'], columns=TOTAL['Restrenyiment']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i TOTAL.


In [171]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Restrenyiment'], columns=TOTAL['Restrenyiment']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i TOTAL.


In [172]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Restrenyiment'], columns=TOTAL['Restrenyiment']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en SENSE_PA i TOTAL.


# Historial mèdic/comorbilitats

## Malalties CV

In [173]:
cv_AMB_PA= (AMB_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA:.2f}%")

Percentatge de problemes CV: 5.88%


In [174]:
cv_AMB_PA_MECVV= (AMB_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA_MECVV:.2f}%")

Percentatge de problemes CV: 0.00%


In [175]:
cv_SENSE_PA= (SENSE_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_SENSE_PA:.2f}%")

Percentatge de problemes CV: 25.00%


In [176]:
cv_TOTAL= (TOTAL['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_TOTAL:.2f}%")

Percentatge de problemes CV: 6.00%


#### Comparació distributiva

###### Normalitat i p-valor

In [177]:
contingency_table_12 = pd.crosstab(index=AMB_PA['CV'], columns=AMB_PA_MECVV['CV'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i AMB_PA_MECVV.


In [178]:
contingency_table_13 = pd.crosstab(index=AMB_PA['CV'], columns=SENSE_PA['CV'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i SENSE_PA.


In [179]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['CV'], columns=SENSE_PA['CV'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [180]:
contingency_table_14 = pd.crosstab(index=AMB_PA['CV'], columns=TOTAL['CV'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 3.7353515625
P-valor entre AMB_PA i TOTAL: 0.053272454982386926
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i TOTAL.


In [181]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['CV'], columns=TOTAL['CV'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i TOTAL.


In [182]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['CV'], columns=TOTAL['CV'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 3.5555555555555554
P-valor entre SENSE_PA i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en SENSE_PA i TOTAL.


## Malalties del cor

In [183]:
cora_AMB_PA= (AMB_PA['Probl.cor'] == 1.0).mean() * 100
print(f"Percentatge de malalties del cor: {cora_AMB_PA:.2f}%")

Percentatge de malalties del cor: 11.76%


In [184]:
cora_AMB_PA_MECVV= (AMB_PA_MECVV['Probl.cor'] == 1.0).mean() * 100
print(f"Percentatge de malalties del cor: {cora_AMB_PA_MECVV:.2f}%")

Percentatge de malalties del cor: 0.00%


In [185]:
cora_SENSE_PA= (SENSE_PA['Probl.cor'] == 1.0).mean() * 100
print(f"Percentatge de malalties del cor: {cora_SENSE_PA:.2f}%")

Percentatge de malalties del cor: 37.50%


In [186]:
cora_TOTAL= (TOTAL['Probl.cor'] == 1.0).mean() * 100
print(f"Percentatge de malalties del cor: {cora_TOTAL:.2f}%")

Percentatge de malalties del cor: 14.00%


#### Comparació distributiva

##### Normalitat i p-valor 

In [187]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Probl.cor'], columns=AMB_PA_MECVV['Probl.cor'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i AMB_PA_MECVV.


In [188]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Probl.cor'], columns=SENSE_PA['Probl.cor'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i SENSE_PA.


In [189]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Probl.cor'], columns=SENSE_PA['Probl.cor'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [190]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Probl.cor'], columns=TOTAL['Probl.cor'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 8.731388888888889
P-valor entre AMB_PA i TOTAL: 0.0031277815159609333
Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i TOTAL.


In [191]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Probl.cor'], columns=TOTAL['Probl.cor'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i TOTAL.


In [192]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Probl.cor'], columns=TOTAL['Probl.cor'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de malalties del cor en SENSE_PA i TOTAL.


## Malalties neurodegeneratives

In [193]:
nd_AMB_PA= (AMB_PA['Neurodegeneratives'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 0.00%


In [194]:
nd_AMB_PA_MECVV= (AMB_PA_MECVV['Neurodegeneratives'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA_MECVV:.2f}%")

Percentatge de malalties neurodegeneratives: 12.50%


In [195]:
nd_SENSE_PA= (SENSE_PA['Neurodegeneratives'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_SENSE_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 12.50%


In [196]:
nd_TOTAL= (TOTAL['Neurodegeneratives'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_TOTAL:.2f}%")

Percentatge de malalties neurodegeneratives: 4.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [197]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Neurodegeneratives'], columns=AMB_PA_MECVV['Neurodegeneratives'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i AMB_PA_MECVV.


In [198]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Neurodegeneratives'], columns=SENSE_PA['Neurodegeneratives'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i SENSE_PA.


In [199]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Neurodegeneratives'], columns=SENSE_PA['Neurodegeneratives'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [200]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Neurodegeneratives'], columns=TOTAL['Neurodegeneratives'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i TOTAL.


In [201]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Neurodegeneratives'], columns=TOTAL['Neurodegeneratives'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 1.469387755102041
P-valor entre AMB_PA_MECVV i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i TOTAL.


In [202]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Neurodegeneratives'], columns=TOTAL['Neurodegeneratives'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en SENSE_PA i TOTAL.


## Diabetis mellitus

In [203]:
dm_AMB_PA= (AMB_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetis mellitus: {dm_AMB_PA:.2f}%")

Percentatge de diabetis mellitus: 23.53%


In [204]:
dm_AMB_PA_MECVV= (AMB_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetis mellitus: {dm_AMB_PA_MECVV:.2f}%")

Percentatge de diabetis mellitus: 25.00%


In [205]:
dm_SENSE_PA= (SENSE_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetis mellitus: {dm_SENSE_PA:.2f}%")

Percentatge de diabetis mellitus: 25.00%


In [206]:
dm_TOTAL= (TOTAL['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetis mellitus: {dm_TOTAL:.2f}%")

Percentatge de diabetis mellitus: 26.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [207]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Neurodegeneratives'], columns=AMB_PA_MECVV['Neurodegeneratives'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i AMB_PA_MECVV.


In [208]:
contingency_table_13 = pd.crosstab(index=AMB_PA['DM'], columns=SENSE_PA['DM'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i SENSE_PA.


In [209]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['DM'], columns=SENSE_PA['DM'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [210]:
contingency_table_14 = pd.crosstab(index=AMB_PA['DM'], columns=TOTAL['DM'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i TOTAL.


In [211]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['DM'], columns=TOTAL['DM'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i TOTAL.


In [212]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['DM'], columns=TOTAL['DM'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 3.5555555555555554
P-valor entre SENSE_PA i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de diabetis mellitus en SENSE_PA i TOTAL.


## Hepatopaties

In [213]:
hp_AMB_PA= (AMB_PA['Hepatopaties'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA:.2f}%")

Percentatge de hepatopaties: 11.76%


In [214]:
hp_AMB_PA_MECVV= (AMB_PA_MECVV['Hepatopaties'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA_MECVV:.2f}%")

Percentatge de hepatopaties: 12.50%


In [215]:
hp_SENSE_PA= (SENSE_PA['Hepatopaties'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_SENSE_PA:.2f}%")

Percentatge de hepatopaties: 0.00%


In [216]:
hp_TOTAL= (TOTAL['Hepatopaties'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_TOTAL:.2f}%")

Percentatge de hepatopaties: 6.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [217]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Hepatopaties'], columns=AMB_PA_MECVV['Hepatopaties'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.9600000000000002
P-valor entre AMB_PA i AMB_PA_MECVV: 0.32718687779030275
No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i AMB_PA_MECVV.


In [218]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Hepatopaties'], columns=SENSE_PA['Hepatopaties'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i SENSE_PA.


In [219]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Hepatopaties'], columns=SENSE_PA['Hepatopaties'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [220]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Hepatopaties'], columns=TOTAL['Hepatopaties'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 8.731388888888889
P-valor entre AMB_PA i TOTAL: 0.0031277815159609333
Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i TOTAL.


In [221]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Hepatopaties'], columns=TOTAL['Hepatopaties'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 1.469387755102041
P-valor entre AMB_PA_MECVV i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i TOTAL.


In [222]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Hepatopaties'], columns=TOTAL['Hepatopaties'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de hepatopaties en SENSE_PA i TOTAL.


## Neoplàsies

In [223]:
neopl_AMB_PA= (AMB_PA['Neoplàsies'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsies: {neopl_AMB_PA:.2f}%")

Percentatge de neoplàsies: 0.00%


In [224]:
neopl_AMB_PA_MECVV= (AMB_PA_MECVV['Neoplàsies'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsies: {neopl_AMB_PA_MECVV:.2f}%")

Percentatge de neoplàsies: 0.00%


In [225]:
neopl_SENSE_PA= (SENSE_PA['Neoplàsies'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsies: {neopl_SENSE_PA:.2f}%")

Percentatge de neoplàsies: 12.50%


In [226]:
neopl_TOTAL= (TOTAL['Neoplàsies'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsies: {neopl_TOTAL:.2f}%")

Percentatge de neoplàsies: 6.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [227]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Neoplàsies'], columns=AMB_PA_MECVV['Neoplàsies'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i AMB_PA_MECVV.


In [228]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Neoplàsies'], columns=SENSE_PA['Neoplàsies'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i SENSE_PA.


In [229]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Neoplàsies'], columns=SENSE_PA['Neoplàsies'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [230]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Neoplàsies'], columns=TOTAL['Neoplàsies'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i TOTAL.


In [231]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Neoplàsies'], columns=TOTAL['Neoplàsies'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i TOTAL.


In [232]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Neoplàsies'], columns=TOTAL['Neoplàsies'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de neoplàsies en SENSE_PA i TOTAL.


## Malaltia crònica renal (creatinina > 1.5mg/dL)

Donat que aquesta columna és de tipus objecte, primer la passaré a tipus numèric per poder operar amb ella

In [233]:
AMB_PA.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(AMB_PA['Creatinina >1.5 binària'], errors='coerce')
AMB_PA_MECVV.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(AMB_PA_MECVV['Creatinina >1.5 binària'], errors='coerce')
SENSE_PA.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(SENSE_PA['Creatinina >1.5 binària'], errors='coerce')
TOTAL.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(TOTAL['Creatinina >1.5 binària'], errors='coerce')

In [234]:
ecr_AMB_PA= (AMB_PA['Creatinina >1.5 binària'] == 1.0).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [235]:
ecr_AMB_PA_MECVV= (AMB_PA_MECVV['Creatinina >1.5 binària'] == 1.0).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA_MECVV:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [236]:
ecr_SENSE_PA= (SENSE_PA['Creatinina >1.5 binària'] == 1.0).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_SENSE_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [237]:
ecr_TOTAL= (TOTAL['Creatinina >1.5 binària'] == 1.0).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_TOTAL:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 6.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [238]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Creatinina >1.5 binària'], columns=AMB_PA_MECVV['Creatinina >1.5 binària'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i AMB_PA_MECVV.


In [239]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Creatinina >1.5 binària'], columns=SENSE_PA['Creatinina >1.5 binària'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de  malaltia crònica renal amb creatinina >1.5 en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de  malaltia crònica renal amb creatinina >1.5 en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de  malaltia crònica renal amb creatinina >1.5 en AMB_PA i SENSE_PA.


In [240]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Creatinina >1.5 binària'], columns=SENSE_PA['Creatinina >1.5 binària'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [241]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Creatinina >1.5 binària'], columns=TOTAL['Creatinina >1.5 binària'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i TOTAL.


In [242]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Creatinina >1.5 binària'], columns=TOTAL['Creatinina >1.5 binària'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i TOTAL.


In [243]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Creatinina >1.5 binària'], columns=TOTAL['Creatinina >1.5 binària'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en SENSE_PA i TOTAL.


## Fallida renal aguda (o insuficiència renal aguda)

In [244]:
ira_AMB_PA= (AMB_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA:.2f}%")

Percentatge de fallida renal aguda: 23.53%


In [245]:
ira_AMB_PA_MECVV= (AMB_PA_MECVV['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA_MECVV:.2f}%")

Percentatge de fallida renal aguda: 50.00%


In [246]:
ira_SENSE_PA= (SENSE_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_SENSE_PA:.2f}%")

Percentatge de fallida renal aguda: 50.00%


In [247]:
ira_TOTAL= (TOTAL['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_TOTAL:.2f}%")

Percentatge de fallida renal aguda: 26.00%


#### Comparació distributiva

##### Normalitat i p-valor 

In [248]:
contingency_table_12 = pd.crosstab(index=AMB_PA['ARF'], columns=AMB_PA_MECVV['ARF'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.6666666666666665
P-valor entre AMB_PA i AMB_PA_MECVV: 0.10247043485974942
No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i AMB_PA_MECVV.


In [249]:
contingency_table_13 = pd.crosstab(index=AMB_PA['ARF'], columns=SENSE_PA['ARF'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i SENSE_PA.


In [250]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['ARF'], columns=SENSE_PA['ARF'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [251]:
contingency_table_14 = pd.crosstab(index=AMB_PA['ARF'], columns=TOTAL['ARF'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i TOTAL.


In [252]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['ARF'], columns=TOTAL['ARF'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i TOTAL.


In [253]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['ARF'], columns=TOTAL['ARF'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.5
P-valor entre SENSE_PA i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions de fallida renal aguda en SENSE_PA i TOTAL.


## Marejos/síncope/lipotímia/reacció vasovagal

In [254]:
dizsyn_AMB_PA= (AMB_PA['Marejos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [255]:
dizsyn_AMB_PA_MECVV= (AMB_PA_MECVV['Marejos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA_MECVV:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [256]:
dizsyn_SENSE_PA= (SENSE_PA['Marejos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_SENSE_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [257]:
dizsyn_TOTAL= (TOTAL['Marejos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_TOTAL:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 2.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [258]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Marejos'], columns=AMB_PA_MECVV['Marejos'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i AMB_PA_MECVV.


In [259]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Marejos'], columns=SENSE_PA['Marejos'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i SENSE_PA.


In [260]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Marejos'], columns=SENSE_PA['Marejos'])
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [261]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Marejos'], columns=TOTAL['Marejos'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA i TOTAL.


In [262]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Marejos'], columns=TOTAL['Marejos'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA_MECVV i TOTAL.


In [263]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Marejos'], columns=TOTAL['Marejos'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en SENSE_PA i TOTAL.


## VIH

In [264]:
vih_AMB_PA= (AMB_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA:.2f}%")

Percentatge de VIH: 0.00%


In [265]:
vih_AMB_PA_MECVV= (AMB_PA_MECVV['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA_MECVV:.2f}%")

Percentatge de VIH: 0.00%


In [266]:
vih_SENSE_PA= (SENSE_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_SENSE_PA:.2f}%")

Percentatge de VIH: 0.00%


In [267]:
vih_TOTAL= (TOTAL['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_TOTAL:.2f}%")

Percentatge de VIH: 0.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [268]:
contingency_table_12 = pd.crosstab(index=AMB_PA['VIH'], columns=AMB_PA_MECVV['VIH'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i AMB_PA_MECVV.


In [269]:
contingency_table_13 = pd.crosstab(index=AMB_PA['VIH'], columns=SENSE_PA['VIH'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i SENSE_PA.


In [270]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['VIH'], columns=SENSE_PA['VIH'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [271]:
contingency_table_14 = pd.crosstab(index=AMB_PA['VIH'], columns=TOTAL['VIH'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i TOTAL.


In [272]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['VIH'], columns=TOTAL['VIH'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i TOTAL.


In [273]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['VIH'], columns=TOTAL['VIH'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de VIH en SENSE_PA i TOTAL.


## Psicosi

In [274]:
psi_AMB_PA= (AMB_PA['Psicosi'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {vih_AMB_PA:.2f}%")

Percentatge de psicosi: 0.00%


In [275]:
psi_AMB_PA_MECVV= (AMB_PA_MECVV['Psicosi'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_AMB_PA_MECVV:.2f}%")

Percentatge de psicosi: 0.00%


In [276]:
psi_SENSE_PA= (SENSE_PA['Psicosi'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_SENSE_PA:.2f}%")

Percentatge de psicosi: 0.00%


In [277]:
psi_TOTAL= (TOTAL['Psicosi'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_TOTAL:.2f}%")

Percentatge de psicosi: 0.00%


#### Comparació distributiva 

##### Normalitat i p-valor

In [278]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Psicosi'], columns=AMB_PA_MECVV['Psicosi'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i AMB_PA_MECVV.


In [279]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Psicosi'], columns=SENSE_PA['Psicosi'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i SENSE_PA.


In [280]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Psicosi'], columns=SENSE_PA['Psicosi'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [281]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Psicosi'], columns=TOTAL['Psicosi'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i TOTAL.


In [282]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Psicosi'], columns=TOTAL['Psicosi'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i TOTAL.


In [283]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Psicosi'], columns=TOTAL['Psicosi'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en SENSE_PA i TOTAL.


## Deficiències nutricionals

In [284]:
nutri_AMB_PA= (AMB_PA['Def.nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [285]:
nutri_AMB_PA_MECVV= (AMB_PA_MECVV['Def.nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA_MECVV:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [286]:
nutri_SENSE_PA= (SENSE_PA['Def.nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_SENSE_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [287]:
nutri_TOTAL= (TOTAL['Def.nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_TOTAL:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


#### Comparació distributiva

##### Normalitat i p-valor

In [288]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Def.nutri'], columns=AMB_PA_MECVV['Def.nutri'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i AMB_PA_MECVV.


In [289]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Def.nutri'], columns=SENSE_PA['Def.nutri'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i SENSE_PA.


In [290]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Def.nutri'], columns=SENSE_PA['Def.nutri'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [291]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Def.nutri'], columns=TOTAL['Def.nutri'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i TOTAL.


In [292]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Def.nutri'], columns=TOTAL['Def.nutri'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i TOTAL.


In [293]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Def.nutri'], columns=TOTAL['Def.nutri'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en SENSE_PA i TOTAL.
